In [2]:
from post_parser_record import PostParserRecord
post_reader = PostParserRecord("Posts_Coffee.xml")
!pip3 install scipy
!pip3 install nltk
from scipy import spatial
from scipy.spatial import distance
import nltk
nltk.download('punkt')
nltk.download('stopwords')


[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\nicdr_nactbrj\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [27]:
#creating index
import re
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

stop_words = stopwords.words('english')
stop_words.append('p')

#creates tokens
def create_tokens_for_text(text):
  temp_tokens = word_tokenize(re.sub(r'[^\w\s]', ' ', text))
  tokens = [i for i in temp_tokens if i not in stop_words]
  return tokens

#finds and returns all terms in document
def get_list_terms():
    list_terms = []
    for answer_id in post_reader.map_just_answers:
      answer = post_reader.map_just_answers[answer_id]
      text = answer.body.lower().strip()
      split_words = create_tokens_for_text(text)
      list_terms.extend(split_words)

    for question_id in post_reader.map_questions:
      question = post_reader.map_questions[question_id]
      title = question.title.lower().strip()
      body = question.body.lower().strip()
      text = title + " " + body
      split_words = create_tokens_for_text(text)
      list_terms.extend(split_words)
    return list_terms

list_terms = get_list_terms()
print("term list created")
#print(str(len(list_terms)))
#print(str(len(set(list_terms))))

temp_dic = {}
count = 0

#creates a dictionary of the terms with the position in the index
for term in set(list_terms):
  if term not in set(temp_dic):
    temp_dic[term] = count
    count +=1

######################################Document############################################
dic_doc_vector = {}
query_results = {}

#creates document vector for answers
for answer_id in post_reader.map_just_answers:
  answer = post_reader.map_just_answers[answer_id]
  doc_vector = [0] *len(temp_dic.keys())
  text = answer.body.lower().strip()
  filtered_words = create_tokens_for_text(text)
  for token in filtered_words:
    if token in temp_dic:
      index = temp_dic[token]
      doc_vector[index] += 1
  dic_doc_vector[answer_id] = doc_vector

#creates document vector for questions
for question_id in post_reader.map_questions:
  question = post_reader.map_questions[question_id]
  doc_vector = [0] *len(temp_dic.keys())
  title = question.title.lower().strip()
  body = question.body.lower().strip()
  text = title + " " + body
  filtered_words = create_tokens_for_text(text)
  for token in filtered_words:
    if token in temp_dic:
      index = temp_dic[token]
      doc_vector[index] += 1
  dic_doc_vector[question_id] = doc_vector
######################################Document############################################

######################################querys############################################
dic_query_vector = {}
querys = ["espresso","turkish coffee","making a decaffeinated coffee","can I use the same coffee grounds twice?"]

#marks query for printing qurel or runfile
query_num = 0

#creates document vector for questions
for temp_query in querys:
  temp_query = temp_query.lower().strip()
  temp_tokens = word_tokenize(re.sub(r'[^\w\s]', '', temp_query))
  query_tokens = [i for i in temp_tokens if i not in stop_words]
  print("search for: "+str(temp_query))

  #creates vector for query of answers
  for answer_id in post_reader.map_just_answers:
    answer = post_reader.map_just_answers[answer_id]
    query_vector = [0] *len(temp_dic.keys())
    text = answer.body.lower().strip()
    filtered_words = create_tokens_for_text(text)
    for token in query_tokens:
      if token in temp_dic:
        index = temp_dic[token]
        query_vector[index] += 1
    dic_query_vector[answer_id] = query_vector
    result = 1 - distance.cosine(query_vector, dic_doc_vector[answer_id])
    query_results[answer_id] = result

  #creates vector for query of questions
  for question_id in post_reader.map_questions:
    question = post_reader.map_questions[question_id]
    query_vector = [0] *len(temp_dic.keys())
    title = question.title.lower().strip()
    body = question.body.lower().strip()
    text = title + " " + body
    filtered_words = create_tokens_for_text(text)
    for token in query_tokens:
      if token in temp_dic:
        index = temp_dic[token]
        query_vector[index] += 1
    dic_query_vector[question_id] = query_vector
    result = 1 - distance.cosine(query_vector, dic_doc_vector[question_id])
    query_results[question_id] = result
######################################querys############################################

######################################print############################################
  result_set = sorted(query_results.items(), key=lambda x: x[1], reverse=True)
  i = 0
  query_num += 1
  for items in result_set:
    if i <= 4:
      print("Document: "+str(items[0])+"\tScore: "+str(items[1]))
      #to print qrel
      #print("A"+str(query_num)+"\t0\t"+str(items[0])+"\t")
      #to print run_file
      #print("A"+str(query_num)+"\tQ0\t"+str(items[0])+"\t"+str(i+1)+"\t"+str("%.02f"%items[1])+"\tSTANDARD")
      i += 1
    else:
      break
######################################print############################################



term list created
search for: espresso
Document: 2766	Score: 0.6614378277661477
Document: 4175	Score: 0.6401843996644798
Document: 26	Score: 0.6299407883487119
Document: 5528	Score: 0.6155870112510924
Document: 3168	Score: 0.588348405414552
search for: turkish coffee
Document: 5094	Score: 0.7715167498104595
Document: 2522	Score: 0.7252406676228422
Document: 3074	Score: 0.7071067811865475
Document: 2379	Score: 0.6832312780114154
Document: 5095	Score: 0.6415415518117004
search for: making a decaffeinated coffee
Document: 120	Score: 0.560448538317805
Document: 4193	Score: 0.501280411827603
Document: 3293	Score: 0.5
Document: 204	Score: 0.492365963917331
Document: 2158	Score: 0.49170515840695195
search for: can i use the same coffee grounds twice?
Document: 2683	Score: 0.6565321642986127
Document: 1749	Score: 0.6002450479987808
Document: 3258	Score: 0.5454545454545454
Document: 5121	Score: 0.5388159060803247
Document: 2609	Score: 0.5151021148075838


In [20]:
num = 1.238475930
print("%.02f"%num)

1.24
